In [1]:
from clickhouse_connect import get_client
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

client = get_client(
    host="chdb.tradegdb.com",
    port=8123,
    username="wanghao",
    password="T8yK1pR4bM9cH2Va"
)

In [6]:
def get_order_cancel(code: str, date: str = '2025-03-03'):
    # get Order Data
    order_sql = f"""
    SELECT 
        time_int,
        price,
        volume,
        side,
        type,
        trade_flag,
        ordno
    FROM stock_base.zb 
    WHERE code = '{code}'
        AND date = '{date}'
        AND date_time >= '{date} 09:15:00'
        AND date_time <= '{date} 09:25:00'
        AND type IS NOT NULL  -- 挂单数据
        AND price IS NOT NULL
        AND volume IS NOT NULL
    ORDER BY price
    """
    orders_df = client.query_df(order_sql)

    cancel_sql = f"""
    SELECT 
        price,
        trade_flag,
        bidno,
        askno
    FROM stock_base.zb 
    WHERE code = '{code}'
        AND date = '{date}'
        AND date_time >= '{date} 09:15:00'
        AND date_time <= '{date} 09:20:00'  -- 撤单只在09:20前
        AND trade_flag = '4'  -- 撤单标识
    ORDER BY date_time
    """
    cancel_df = client.query_df(cancel_sql)
    
    return orders_df,cancel_df

def get_all_data(code: str, date: str = '2025-03-03'):
    sql = f"""
    SELECT 
        time_int,
        price,
        volume,
        side,
        ordno
    FROM stock_base.zb 
    WHERE code = '{code}'
        AND date = '{date}'
        AND date_time >= '{date} 09:15:00'
        AND date_time <= '{date} 09:25:00'
        AND type IS NOT NULL  -- 挂单数据
        AND trade_flag != '4'  -- 撤单标识
        AND price IS NOT NULL
        AND volume IS NOT NULL
    ORDER BY price
    """
    df = client.query_df(sql)
    return df

In [7]:
orders_df,cancel_df = get_order_cancel('000001')
print(orders_df)
print(cancel_df)

      time_int  price  volume     side     type trade_flag   ordno
0     33300130   0.00     100  b'\x00'  b'\x00'       b'4'   21677
1     33300280   0.00    5000  b'\x00'  b'\x00'       b'4'   61819
2     33300320   0.00   13000  b'\x00'  b'\x00'       b'4'   66523
3     33300320   0.00     100  b'\x00'  b'\x00'       b'4'   67371
4     33300360   0.00     100  b'\x00'  b'\x00'       b'4'   72506
...        ...    ...     ...      ...      ...        ...     ...
6475  33881140  12.68    1000     b'S'     b'2'    b'\x00'  416813
6476  33887390  12.68     100     b'S'     b'2'    b'\x00'  422211
6477  33888310  12.68     200     b'B'     b'2'    b'\x00'  423233
6478  33892900  12.68    1000     b'S'     b'2'    b'\x00'  428717
6479  33897300  12.68     900     b'B'     b'2'    b'\x00'  436958

[6480 rows x 7 columns]
     price trade_flag   bidno   askno
0      0.0       b'4'    4779       0
1      0.0       b'4'   55282       0
2      0.0       b'4'   66282       0
3      0.0       b'

In [8]:
df = get_all_data('000001')
print(df)

      time_int  price  volume  side   ordno
0     33300000  10.38   10300  b'B'       5
1     33300000  10.38     100  b'B'       8
2     33300000  10.38     100  b'B'      12
3     33300000  10.38    1100  b'B'      15
4     33300000  10.38   10100  b'B'      21
...        ...    ...     ...   ...     ...
5883  33881140  12.68    1000  b'S'  416813
5884  33887390  12.68     100  b'S'  422211
5885  33888310  12.68     200  b'B'  423233
5886  33892900  12.68    1000  b'S'  428717
5887  33897300  12.68     900  b'B'  436958

[5888 rows x 5 columns]


In [12]:
buy_orders = df[df['side'] == b'B'].copy()
sell_orders = df[df['side'] == b'S'].copy()

# 非买卖单
other_orders = df[(df['side'] != b'B') & (df['side'] != b'S')]

In [13]:
# 买盘：价格从高到低排序（买方愿意出高价优先）
buy_orders_sorted = buy_orders.sort_values('price', ascending=False)

# 卖盘：价格从低到高排序（卖方愿意低价卖优先）
sell_orders_sorted = sell_orders.sort_values('price', ascending=True)

print("买盘排序后（价格高→低）:")
print(buy_orders_sorted[['price', 'volume']].head(100))

print("\n卖盘排序后（价格低→高）:")
print(sell_orders_sorted[['price', 'volume']].head(100))

买盘排序后（价格高→低）:
      price  volume
5862  12.68     100
5863  12.68   14800
5815  12.68     200
5864  12.68     200
5869  12.68     100
...     ...     ...
2907  11.53     100
2926  11.53    2000
2909  11.53    4700
2925  11.53     100
2895  11.53     100

[100 rows x 2 columns]

卖盘排序后（价格低→高）:
      price  volume
19    10.38     100
22    10.38    4400
287   10.38    2000
297   10.38     600
298   10.38   19000
...     ...     ...
2937  11.53     500
3007  11.53     900
3012  11.53    2000
3017  11.53    6800
2956  11.53    2000

[100 rows x 2 columns]


In [14]:
def calculate_opening_price(buy_orders, sell_orders):
    """
    计算集合竞价开盘价
    buy_orders: 买盘DataFrame，已按价格降序排列
    sell_orders: 卖盘DataFrame，已按价格升序排列
    """
    # 1. 获取所有可能的价格
    buy_prices = buy_orders['price'].unique()
    sell_prices = sell_orders['price'].unique()
    all_prices = sorted(set(list(buy_prices) + list(sell_prices)))
    
    max_volume = 0
    best_price = None
    
    # 2. 试算每个价格
    for price in all_prices:
        # 买方：愿意以>=当前价格买的累计量
        buy_volume = buy_orders[buy_orders['price'] >= price]['volume'].sum()
        
        # 卖方：愿意以<=当前价格卖的累计量
        sell_volume = sell_orders[sell_orders['price'] <= price]['volume'].sum()
        
        # 可成交量 = min(买方可买量, 卖方可卖量)
        volume = min(buy_volume, sell_volume)
        
        # 3. 找最大成交量
        if volume > max_volume:
            max_volume = volume
            best_price = price
    
    return best_price, max_volume

# 使用
opening_price, volume = calculate_opening_price(buy_orders_sorted, sell_orders_sorted)
print(f"计算的开盘价: {opening_price}")
print(f"成交量: {volume}")

计算的开盘价: 11.52
成交量: 454500


In [15]:
# 从日线表获取真实开盘价
def get_real_opening_price(code, date:str = '2025-03-03'):
    sql = f"""
    SELECT open 
    FROM stock_base.daily 
    WHERE code = '{code}' AND date = '{date}'
    LIMIT 1
    """
    
    try:
        df = client.query_df(sql)
        if not df.empty:
            return float(df.iloc[0]['open'])
    except Exception as e:
        print(f"查询失败: {e}")
    
    return None

# 使用
real_price = get_real_opening_price('000001', '2025-03-03')
print(f"真实开盘价: {real_price}")
print(f"计算开盘价: {opening_price}")
print(f"误差: {abs(opening_price - real_price) if real_price else '无数据'}")

真实开盘价: 11.52
计算开盘价: 11.52
误差: 0.0
